In [1]:
%matplotlib inline
import numpy as np
import pandas as pd


In [2]:
def generate_data_matrix(spatial_size, temporal_size, miss_ratio):
    data_matrix = np.random.randint(1,5,size=(spatial_size,temporal_size))
    origin = data_matrix.copy()
    k = round(spatial_size*temporal_size*miss_ratio)
    ri = np.random.choice(spatial_size*temporal_size,k,replace=False)
    data_matrix.flat[ri] = 0
    return origin,data_matrix

In [5]:
def is_diagonal(matrix):
    result = matrix.T == matrix
    if type(result) == bool:
        return result
    else:
        return result.all()

In [6]:
def get_distance_matrix(data_matrix,dis_loc=None):
    spatial_length = data_matrix.shape[0]
    # print(spatial_length)
    dis_matrix = np.random.rand(spatial_length,spatial_length)
    dis_matrix += dis_matrix.T
    np.fill_diagonal(dis_matrix,0)
    return dis_matrix

In [10]:
def get_time_interval_matrix(data_matrix):
    temporal_length = data_matrix.shape[1]
    print(temporal_length)
    ti_matrix = np.random.rand(temporal_length,temporal_length)
    

get_time_interval_matrix(dm)

4


In [8]:
origin,dm = generate_data_matrix(5,4,0.3)
print(origin)
print(dm)
sum = 0 
for i in dm.flatten():
    if i == 0:
        sum +=1
print(sum)

[[2 2 3 1]
 [1 1 2 2]
 [1 4 4 1]
 [1 1 4 1]
 [3 1 3 1]]
[[2 2 3 1]
 [0 1 2 2]
 [1 4 0 0]
 [0 1 0 1]
 [0 1 3 1]]
6


In [61]:
# Global Temporal View -SES(Simple Exponential Smoothing)

def ses(data_matrix,beta=0.5,time_interval_array=None):
    return_matrix = data_matrix.copy().astype(float)
    
    spatial_length,temporal_length = data_matrix.shape
    for i in range(spatial_length):
        tmp = data_matrix[i]
        # print(tmp)
        zero_index = np.where(tmp == 0)[0]
        true_index = np.where(tmp != 0)[0]
        res_array = np.zeros_like(zero_index,dtype=float)
        # print('-')
        for j,mis in enumerate(zero_index):
            # print('--')
            time_interval = abs(true_index-mis)
            # print(time_interval)
            # coef_array = time_interval
            coef_array = np.power(1-beta,time_interval-1)*beta
            res_array[j] = np.dot(coef_array,tmp[true_index]) / (np.sum(coef_array))
            # print('res',res_array[j])
        # print(zero_index)
        return_matrix[i][zero_index] = res_array
    return return_matrix

origin,dm = generate_data_matrix(5,4,0.3)
ses(dm)-dm


array([[0.        , 0.        , 1.6       , 0.        ],
       [0.        , 1.4       , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 3.42857143],
       [1.57142857, 0.        , 0.        , 0.        ],
       [1.        , 1.        , 0.        , 0.        ]])

In [ ]:
# Global Spatial View - IDW (Inverse distance weighting)
# Global means using all sensors in the sensor network
# Local means using a subset of sensors in the sensor network

def idw(data_matrix,alpha=1,distance_matrix=None):
    return_matrix = data_matrix.copy().astype(float)
    # return_matrix = data_matrix
    if distance_matrix is not None:
        spaital_length,temporal_length = data_matrix.shape

        for i in range(temporal_length):
            tmp = data_matrix.T[i]
            zero_index = np.where(tmp == 0)[0]
            true_index = np.where(tmp != 0)[0]
            res_array = np.zeros_like(zero_index,dtype=float)
            for j,mis in enumerate(zero_index):                
                dis_array = np.zeros_like(true_index,dtype=np.float)
                for ii,solid in enumerate(true_index):
                    dis_array[ii] = distance_matrix[mis][solid]
                dis_array = np.power(dis_array,-alpha)
                res_array[j] = np.dot(dis_array,tmp[true_index]) / np.sum(dis_array)

            return_matrix.T[i][zero_index] = res_array
    else:
        pass
    return return_matrix

origin,dm = generate_data_matrix(5,4,0.3)
ds_matrix = get_distance_matrix(dm,0)
res = idw(dm,distance_matrix=ds_matrix)
print(res-dm)
